In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import NearestNeighbors
import pandas as pd

In [28]:
data = pd.read_csv('genres_v2.csv')

/var/folders/cc/c2h4hf8x1rq2d8qclpl8dwdw0000gn/T/ipykernel_50101/2411600261.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('genres_v2.csv')


In [29]:
# Preprocessing the data
# Assuming 'genre' as the target variable for classification
X = data.select_dtypes(include=['float64', 'int64']).dropna()  # Selecting numerical features
y = data.loc[X.index, 'genre']  # Corresponding genres

In [30]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [31]:
# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [38]:
# give predictions for input based on the model
def get_recommendations(input):
    # Selecting a random song from the dataset that has complete feature data
    random_song = data.dropna(subset=data.select_dtypes(include=['float64', 'int64']).columns).sample()

    # Extracting the features of the random song
    random_song_features = random_song.select_dtypes(include=['float64', 'int64'])
    nn = NearestNeighbors(n_neighbors = 6) # get 6 nearest neighbors
    nn.fit(X)
    distances, indices = nn.kneighbors(random_song_features)
    nearest_neighbors = indices[0]
    # Fetching the song names of the nearest neighbors
    recommended_songs = data.iloc[nearest_neighbors]['song_name']
    # Preparing the result excluding the random song itself
    recommendations = recommended_songs[recommended_songs != input].tolist()
    return recommendations




In [39]:
# Test recommendations
input = 'Dior - Bonus'
recommendations = get_recommendations(input)
print(recommendations)

AttributeError: 'str' object has no attribute 'select_dtypes'